In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook 

import json
with open('/Users/sofiawang/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api_key'])

In [10]:
!pip install tqdm

In [11]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [15]:
import tmdbsimple as tmdb
tmdb.API_KEY = login['api_key']

In [18]:
YEARS_TO_GET = [2001,2002]

for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    json_filename = f'results_in_progress_{YEARS_TO_GET}.json'

    results = {"YEARS_TO_GET": "year", "data": "your_data_here"}
    with open(json_filename, 'w') as json_file:
        json.dump(results, json_file)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
movies= pd.read_csv("Data/title.basics.tsv.gz", sep='\t', low_memory=False)
movies

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
10017006,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2009,\N,\N,"Action,Drama,Family"
10017007,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
10017008,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
10017009,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [17]:
filtered_movies = movies[(movies['start_year'] == 2001) & (movies['start_year'] == 2002)]

NameError: name 'movies' is not defined

In [ ]:
import requests

def imdb_to_tmdb(imdb_id):
    mapping_url = f'https://api.themoviedb.org/3/find/{imdb_id}?api_key=YOUR_API_KEY&external_source=imdb_id'
    response = requests.get(mapping_url)
    data = response.json()

    tmdb_id = data.get('movie_results', [{}])[0].get('id')

    return tmdb_id

def get_movie_info(imdb_id):
    tmdb_id = imdb_to_tmdb(imdb_id)
    
imdb_id_avengers = 'tt0848228'
imdb_id_notebook = 'tt0332280'

movie_info_avengers = get_movie_info(imdb_id_avengers)
movie_info_notebook = get_movie_info(imdb_id_notebook)

print("The Avengers Movie Info:")
print(movie_info_avengers)

print("\nThe Notebook Movie Info:")
print(movie_info_notebook)


In [ ]:
import glob

tmdb_files = sorted(glob.glob("Data/final_tmdb_data*.csv.gz"))
tmdb_files

In [ ]:
df = pd.concat([pd.read_csv(f) for f in tmdb_files] )
df

In [ ]:
df = pd.concat([pd.read_csv(f, lineterminator='\n') for f in files] )